# Readme
1. This is the code for predicting aesthetic scores or histograms using the method in the paper:
Jin, Bin, Maria V. Ortiz Segovia, and Sabine Süsstrunk. "Image aesthetic predictors based on weighted CNNs." Image Processing (ICIP), 2016 IEEE International Conference on. IEEE, 2016.

2. To use the code, you need to install the following packages:
Theano(>=0.9), Keras(>=1.0), numpy, opencv

3. To run the code: please download the weights from our project webpage.
There are two weights hdf5 files, one is for predicting the aesthetics score (1 value), the other one is for predicting the histogram (10 bins). Both of them are ~500 MB since the network architecture is VGG16.
Modify the "outputflag" parameter to change between score prediction or histogram prediction. 

----------Created by Bin Jin. Nov. 16th, 2016.

#### Load the packages

In [ ]:
from __future__ import division

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import cv2, os, random, time

## set global parameters: backend and image_dim_ordering for keras, and the gpu configuration
import os
os.environ['KERAS_BACKEND'] = 'theano' # or 'tensorflow'
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
from keras import backend as K
K.set_image_dim_ordering('th')

from keras.optimizers import SGD
from keras.utils import np_utils, generic_utils
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.regularizers import l2


#### Define the network architecture

In [ ]:
def AestheticNet(outputflag='score',weight_decay=0.0005, weights_path=None):
    model = Sequential()
    model.add(ZeroPadding2D((1,1),input_shape=(3,224,224)))
    model.add(Convolution2D(64, 3, 3, activation='relu',init='he_normal',W_regularizer=l2(weight_decay)))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(64, 3, 3, activation='relu',init='he_normal',W_regularizer=l2(weight_decay)))
    model.add(MaxPooling2D((2,2), strides=(2,2))) # The receptive filed is 7*7 here

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu',init='he_normal',W_regularizer=l2(weight_decay)))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu',init='he_normal',W_regularizer=l2(weight_decay)))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu',init='he_normal',W_regularizer=l2(weight_decay)))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu',init='he_normal',W_regularizer=l2(weight_decay)))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu',init='he_normal',W_regularizer=l2(weight_decay)))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu',init='he_normal',W_regularizer=l2(weight_decay)))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu',init='he_normal',W_regularizer=l2(weight_decay)))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu',init='he_normal',W_regularizer=l2(weight_decay)))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu',init='he_normal',W_regularizer=l2(weight_decay)))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu',init='he_normal',W_regularizer=l2(weight_decay)))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu',init='he_normal',W_regularizer=l2(weight_decay)))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(Flatten())
    model.add(Dense(4096, activation='relu',init='he_normal',W_regularizer=l2(weight_decay)))

    model.add(Dropout(0.5))
    model.add(Dense(4096, activation='relu',init='he_normal',W_regularizer=l2(weight_decay)))

    model.add(Dropout(0.5))
    if outputflag =='score' :
        model.add(Dense(1,init='he_normal',W_regularizer=l2(weight_decay)))
    else:
        model.add(Dense(10, activation='softmax',init='he_normal',W_regularizer=l2(weight_decay)))    

    if weights_path:
        model.load_weights(weights_path)

    return model

#### Load the weights of the network

In [ ]:
outputflag = 'score' ## 'score' for predicting the aesthetic score (1 value),'hist' for predicting the histogram (10 bins)
path_weights = './' ## modify to the path where you put the downloaded weights
weights_file = 'AVA_AestheticNet_' + outputflag + '_ICIP.h5'
model = AestheticNet(outputflag=outputflag,weight_decay=0.0005, weights_path=path_weights + weights_file)

#### Prediction

In [ ]:
path_images = './'
filename = 'example.jpg' ## specify your own image

img = cv2.imread(path_images + filename)
imgn = np.copy(img)
imgn = cv2.resize(imgn,(224,224))
imgn = np.transpose(imgn,[2,0,1])
imgn = np.expand_dims(imgn,axis=0)
score = model.predict(imgn)

img=cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.imshow(img)
plt.show()

print filename + ': ', score


##### code for cropping the input image for aesthetic appealingness

In [ ]:
## function for generating crop:
## Inputs:
##     ---- img: the input image, the format should be (height, width, 3)
##     ---- model: the aesthetics prediction model
##     ---- n_crops_attemp: number of random crops to choose from, the more, the slower
##     ---- aespect_ratio_h, aespect_ratio_w: the aespect ratio number for the height and width, the generated crop will
##                                            be in the aespect ratio of: aespect_ratio_h:aespect_ratio_w (e.g. 16:9)
##     ---- length_constrain: constrain the minsize of the crop to be larger than a percentage of the width or height.
##
## Outputs:
##     ---- crop_best: the generated best crop of the input image
##     ---- score_best: the aesthetics score of the returned crop, in range (0,1)

def generate_crop(img, model, n_crops_attemp=250, aespect_ratio_h=9, aespect_ratio_w=16,length_constrain = 0.5):
    
    [h,w,c] = img.shape
    images_batch = np.zeros((n_crops_attemp,3,224,224),np.float32)
    coordinates_batch = np.zeros((n_crops_attemp,4),dtype=np.uint16)
    min_sz_step = int(min(h/aespect_ratio_h, w/aespect_ratio_w))
    
    for crop_counter in range(n_crops_attemp):
        temp = random.randint(int(min_sz_step/2),min_sz_step)
        h_temp =  temp * aespect_ratio_h
        w_temp = temp * aespect_ratio_w
        start_h = random.randint(0,h-h_temp)
        start_w = random.randint(0,w-w_temp)
        end_h = start_h + h_temp
        end_w = start_w + w_temp
        img_crop = img[start_h:end_h,start_w:end_w,:]
        img_crop = cv2.resize(img_crop,(224,224))

        images_batch[crop_counter] = img_crop.transpose((1,2,0))
        coordinates_batch[crop_counter] = [start_h, start_w, end_h,end_w]

    scores_batch = model.predict(preprocess_input(images_batch))[:]
    score_batch_bestidx = np.argmax(scores_batch)
    [start_h,start_w,end_h,end_w] = coordinates_batch[score_batch_bestidx]
    crop_best = img[start_h:end_h,start_w:end_w,:]
    score_best = scores_batch[score_batch_bestidx]
    return crop_best, score_best

In [ ]:
path_images = './'
filename = 'Holiday.jpeg' ## specify your own image

img = cv2.imread(path_images + filename)
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.show()

[crop_best,score_best] = generate_crop(img, model, n_crops_attemp=100)
plt.imshow(cv2.cvtColor(crop_best, cv2.COLOR_BGR2RGB))
plt.show()
print score_best